In [ ]:
import math
import pexpect #pip install pexpect
import re
import string
import math
import random
import subprocess
import time
import os
import numpy as np
file_name='train4'
os.chdir("/home/kaushik/Project/kiss2_pla/"+file_name)   #Provide directory where all necessary kiss2 file is
pattern2 = re.compile(r'\S+')
file_lst=[]
particle_struct1=[]
particle_cost1=[]
random_lst=[]
value_to_be_padded=-1
max_4bit_state=16
no_of_data=1
kissfile =file_name+".kiss2"                 #  Kiss2 file to be converted to pla
plafile =file_name                   #  Modified pla file
outfile =file_name+"_literal.pla"                #  Minimized pla after espresso
with open(file_name+'.kiss2','r') as f:
    for line in f:
        if line[0]!='.':
            match=pattern2.findall(line)
            file_lst.append(match)
        #match = pattern2.findall(line)
        #a.append(match)
        elif line[1]=='i':
            matchi = pattern2.findall(line)
            no_of_input=int(matchi[1])
        elif line[1]=='o':
            matcho = pattern2.findall(line)
            no_of_output=int(matcho[1])
        elif line[1]=='s':
            matchs = pattern2.findall(line)
            no_of_states=int(matchs[1])
            
            
            
no_of_bits=math.ceil(math.log2(int(no_of_states)))            
total_combi=2**no_of_bits;  


with open("script.sh","w+") as f:
    f.write("#!/bin/bash\n")
    for i in range(no_of_data):    
        f.write("read_pla "+plafile+str(i)+'.pla'+"\n")
        f.write("source script.rugged\n")
        f.write("print_stats\n")
    f.write("quit")
    
tint=time.time()

def pad(l, content, width):
    l.extend([content] * (width - len(l)))
    return l

def kiss_pla(assign_states):  
    
    #print(assign_states)
    pattern = re.compile(r'\d+')
    pattern2 = re.compile(r'\S+')
    with open(kissfile,'r') as fkiss:
        for line in fkiss:
            if(line[1]=='i'):
                inputs = int(pattern.findall(line)[0])
            elif(line[1]=='o'):
                outputs = int(pattern.findall(line)[0])
            elif(line[1]=='p'):
                products = int(pattern.findall(line)[0])
            elif(line[1]=='s'):
                states = int(pattern.findall(line)[0])
            else:
                break
    extra = math.ceil(math.log2(int(states)))
    
    for i in range(no_of_data):
        with open(kissfile,'r') as fkiss:
            with open(plafile+str(i)+'.pla','w+') as fpla:
                for line in fkiss:
                    if(line[1]=='i'):
                        fpla.write(".i "+str(inputs+extra)+"\n") # plain copy
                    elif(line[1]=='o'):
                        fpla.write(".o "+str(outputs+extra)+"\n")
                    elif(line[1]=='p'):
                        fpla.write(line)
                    elif(line[1]=='s'):
                        pass            
                    elif(line[0]!='\n'):
                        match = pattern2.findall(line)                    
                        s0=match[0]                    
                        s1 = assign_states[i][int(match[1].lstrip('st'))]                    
                        s2 = assign_states[i][int(match[2].lstrip('st'))]                    
                        s3=match[3]                    
                        fpla.write(str(s0)+str(s1)+" "+str(s2)+str(s3)+"\n")
    sis= "sis -x -f script.sh"+" > "+outfile
    os.system(sis)
    a=[]
    with open(outfile,'r') as f:
        for line in f:
            if line[0:3]=='lit':
                match_new = pattern.findall(line)
                a.append(match_new[0])
        #print(a)        #print(a)
    return(a)




in0=np.zeros([no_of_input,no_of_states])
in1=np.zeros([no_of_input,no_of_states])
out0=np.zeros([no_of_output,no_of_states])
out1=np.zeros([no_of_output,no_of_states])
states=np.zeros([no_of_states,no_of_states])

fan_in_edge_weights=np.zeros([no_of_states,no_of_states])
fan_out_edge_weights=np.zeros([no_of_states,no_of_states])
for i in range(len(file_lst)): ## for each line
    for j in range(no_of_input): ## for each element
        for k in range(no_of_states):
            if file_lst[i][0][j]=='0':            
                if file_lst[i][2][2:]==str(k):
                    in0[j][k]+=1                  
                    
                    
            elif file_lst[i][0][j]=='1':            
                if file_lst[i][2][2:]==str(k):
                    in1[j][k]+=1
    for j in range(no_of_output):
        for k in range(no_of_states):
            if file_lst[i][3][j]=='0':            
                if file_lst[i][1][2:]==str(k):
                    out0[j][k]+=1       ##########multi-d variable stores out=0 for n number of output####            
                    
                    
            elif file_lst[i][3][j]=='1':            
                if file_lst[i][1][2:]==str(k):
                    out1[j][k]+=1
    m=int(file_lst[i][1][2:])
    n=int(file_lst[i][2][2:])
    states[m][n]=states[m][n]+1 ###############stores state transition ##########
inn=np.vstack([in0,in1])
outt=np.vstack([out0,out1])

############ calculating edge weight for fan in and fan out oriented#########
for j in range(no_of_states):
    for k in range(no_of_states):
        if int(j)!=int(k):
            fan_in_edge_weights[j][k]=np.dot(inn.T[j],inn.T[k])+no_of_bits*np.dot(states.T[j],states.T[k])
            fan_out_edge_weights[j][k]=np.dot(outt.T[j],outt.T[k])+(no_of_bits/2)*np.dot(states.T[j],states.T[k])
        
                
print("fan_in_edge_weights",fan_in_edge_weights)
print("fan_out_edge_weights",fan_out_edge_weights)


for k in range(0,no_of_data):    
    rand=random.sample(range(total_combi),no_of_states)
    random_lst.append(rand)
    
    assign_states=[]
    for i in range(0,len(rand)):            
        a="0"+str(no_of_bits)+'b'
        assign_states.append(format(rand[i],a))    
    particle_struct1.append(assign_states)
    #print(particle_struct1)
    #particle_cost1.append(p)
particle_cost1 =kiss_pla(particle_struct1)
#print("random_lst",random_lst)
total_hamming_dist=[]
list_fanin_sum=[]
list_fanout_sum=[]
for ii in range(len(random_lst)):
    hamming_dist=np.zeros([no_of_states,no_of_states])
    for i in range(no_of_states):
        for j in range(no_of_states):
            hamming_dist[i][j]=random_lst[ii][i]^random_lst[ii][j]
    total_hamming_dist.append(hamming_dist)
    fanin_multi = np.multiply(fan_in_edge_weights, hamming_dist) 
    fanout_multi = np.multiply(fan_out_edge_weights, hamming_dist)
    fanin_sum=np.sum(np.triu(fanin_multi))
    fanout_sum=np.sum(np.triu(fanout_multi))
    list_fanin_sum.append(fanin_sum)
    list_fanout_sum.append(fanout_sum)
#print(no_of_states)  
fan_in_edge_weights_conc = np.zeros((max_4bit_state,max_4bit_state))
fan_out_edge_weights_conc = np.zeros((max_4bit_state,max_4bit_state))
fan_in_edge_weights_conc[:fan_in_edge_weights.shape[0], :fan_in_edge_weights.shape[1]] = fan_in_edge_weights
#print(z)
fan_out_edge_weights_conc[:fan_out_edge_weights.shape[0], :fan_out_edge_weights.shape[1]] = fan_out_edge_weights
print(fan_in_edge_weights_conc)
print(fan_out_edge_weights_conc)
tfinal=time.time()
#print("total_hamming_dist",total_hamming_dist)


#print("random_lst",random_lst)
#print("list_fanin_sum",list_fanin_sum)
#print("list_fanout_sum",list_fanout_sum)
#print("particle_cost1",particle_cost1)
#print("particle_struct1",particle_struct1)
print("time taken in mins: ",(tfinal-tint)/60)
################padding #################
for i in range(len(random_lst)):
    pad(random_lst[i], value_to_be_padded, total_combi)

#####################
os.chdir("/home/kaushik/Project/kiss2_pla/data_base")    

with open("fan_in_out_data_-1_20k_"+"ios_"+"bbara_"+"train"+".csv","w+") as f:
    f.write("in,out,state,")
    for i in range((total_combi)):
        f.write("st"+str(i)+",")
#     x=1
#     for i in range(0,max_4bit_state-1):
#         for j in range(x,max_4bit_state):
#             #print(fan_in_edge_weights[i][j])       
#             f.write("fanin"+str(i)+str(j)+",")
#         x+=1
#     x=1
#     for i in range(0,max_4bit_state-1):
#         for j in range(x,max_4bit_state):
#             #print(fan_in_edge_weights[i][j])       
#             f.write("fanout"+str(i)+str(j)+",")
#         x+=1
    f.write("faninsum,fanoutsum,literals\n")
    for i in range(no_of_data):
        f.write(str(no_of_input)+","+str(no_of_output)+","+str(no_of_states)+",")
        for item in random_lst[i]:
            f.write(str(item)+",")
#         x=1
#         for ik in range(0,max_4bit_state-1):
#             for j in range(x,max_4bit_state):
#                 f.write(str(int(fan_in_edge_weights_conc[ik][j]))+",")       
#                 #print("fanin"+str(i)+str(j))
#             x+=1
#         x=1
#         for ik in range(0,max_4bit_state-1):
#             for j in range(x,max_4bit_state):
#                 f.write(str(int(fan_out_edge_weights_conc[ik][j]))+",")       
#                 #print("fanin"+str(i)+str(j))
#             x+=1
        f.write(str(int(list_fanin_sum[i]))+","+str(int(list_fanout_sum[i]))+","+particle_cost1[i]+"\n")
    

fan_in_edge_weights [[ 0. 12.  8. 12.]
 [12.  0. 24. 16.]
 [ 8. 24.  0. 24.]
 [12. 16. 24.  0.]]
fan_out_edge_weights [[ 0.  2.  0.  2.]
 [ 2.  0. 20.  8.]
 [ 0. 20.  0. 12.]
 [ 2.  8. 12.  0.]]
[[ 0. 12.  8. 12.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [12.  0. 24. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 8. 24.  0. 24.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [12. 16. 24.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 

In [ ]:
with open("fan_in_out_database.txt","w+") as f:
    for i in range((no_of_states)):
        f.write("st"+str(i)+",")
    f.write("faninsum,fanoutsum,literals\n")
    for i in range(no_of_data):
        for item in random_lst[i]:
            f.write(str(item)+",")
        f.write(str(int(list_fanin_sum[i]))+","+str(int(list_fanout_sum[i]))+","+particle_cost1[i]+"\n")
    

In [6]:
avc=np.zeros((3,4),dtype=fan_in_edge_weights.dtype)
avc.reshape(15,55)

ValueError: cannot reshape array of size 12 into shape (15,55)

In [ ]:
for i in range(len(fan_in_edge_weights)):
    for j in range(len(fan_in_edge_weights[i])):
        print(fan_in_edge_weights[i][j])

In [ ]:
n=4
x=1
for i in range(0,n-1):
    for j in range(x,n):
        print(fan_in_edge_weights[i][j])       
        print("fanin"+str(i)+str(j))
    x+=1
    

In [ ]:
particle_struct1=[]
particle_cost1=[]
random_lst=[]
no_of_data=3
total_combi=2**no_of_bits;
kissfile =file_name+".kiss2"                 #  Kiss2 file to be converted to pla
plafile =file_name                   #  Modified pla file
outfile =file_name+"_literal.pla"                #  Minimized pla after espresso
with open("script.sh","w+") as f:
    f.write("#!/bin/bash\n")
    for i in range(no_of_data):    
        f.write("read_pla "+plafile+str(i)+'.pla'+"\n")
        f.write("source script.rugged\n")
        f.write("print_stats\n")
    f.write("quit")
def kiss_pla(assign_states):  
    
    #print(assign_states)
    pattern = re.compile(r'\d+')
    pattern2 = re.compile(r'\S+')
    with open(kissfile,'r') as fkiss:
        for line in fkiss:
            if(line[1]=='i'):
                inputs = int(pattern.findall(line)[0])
            elif(line[1]=='o'):
                outputs = int(pattern.findall(line)[0])
            elif(line[1]=='p'):
                products = int(pattern.findall(line)[0])
            elif(line[1]=='s'):
                states = int(pattern.findall(line)[0])
            else:
                break
    extra = math.ceil(math.log2(int(states)))
    
    for i in range(no_of_data):
        with open(kissfile,'r') as fkiss:
            with open(plafile+str(i)+'.pla','w+') as fpla:
                for line in fkiss:
                    if(line[1]=='i'):
                        fpla.write(".i "+str(inputs+extra)+"\n") # plain copy
                    elif(line[1]=='o'):
                        fpla.write(".o "+str(outputs+extra)+"\n")
                    elif(line[1]=='p'):
                        fpla.write(line)
                    elif(line[1]=='s'):
                        pass            
                    elif(line[0]!='\n'):
                        match = pattern2.findall(line)                    
                        s0=match[0]                    
                        s1 = assign_states[i][int(match[1].lstrip('st'))]                    
                        s2 = assign_states[i][int(match[2].lstrip('st'))]                    
                        s3=match[3]                    
                        fpla.write(str(s0)+str(s1)+" "+str(s2)+str(s3)+"\n")
    sis= "sis -x -f script.sh"+" > "+outfile
    os.system(sis)
    a=[]
    with open(outfile,'r') as f:
        for line in f:
            if line[0:3]=='lit':
                match_new = pattern.findall(line)
                a.append(match_new[0])
        print(a)        #print(a)
    return(a)
for k in range(0,no_of_data):    
    rand=random.sample(range(total_combi),no_of_states)
    random_lst.append(rand)
    
    assign_states=[]
    for i in range(0,len(rand)):            
        a="0"+str(no_of_bits)+'b'
        assign_states.append(format(rand[i],a))    
    particle_struct1.append(assign_states)
    #print(particle_struct1)
    #particle_cost1.append(p)
particle_cost1 =kiss_pla(particle_struct1)
print(random_lst)

In [ ]:
for ii in range(len(random_lst)):
    hamming_dist=np.zeros([no_of_states,no_of_states])
    for i in range(no_of_states):
        for j in range(no_of_states):
            hamming_dist[i][j]=random_lst[ii][i]^random_lst[ii][j]
    print(hamming_dist)
        

In [ ]:
hamming_dist
out_num = np.multiply(fan_in_edge_weights, hamming_dist) 
out

In [ ]:
particle_cost1

In [ ]:
random_lst

In [ ]:
out_num

In [ ]:
a=np.sum(np.triu(out_num))
#print(out_num)
print(a)

In [ ]:
np.sum(a)